<h1> 2c. Monitoring when using tf.layers </h1>

This [extra] notebook illustrates how to manually add summary operations to nodes, save their values during training and visualize them over training epochs.

In [1]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

Code to read data and compute error is the same as Lab2a.

In [2]:
def read_dataset(filename):
  return pd.read_csv(filename, header=None, names=['pickuplon','pickuplat','dropofflon','dropofflat','passengers','fare_amount'])

df_train = read_dataset('../lab1a/taxi-train.csv')
df_valid = read_dataset('../lab1a/taxi-valid.csv')
df_test = read_dataset('../lab1a/taxi-test.csv')
df_train[:5]

,pickuplon,pickuplat,dropofflon,dropofflat,passengers,fare_amount
0,-73.984162,40.767241,-73.967796,40.752417,1,9.7
1,-74.005099,40.719629,-74.010202,40.719718,3,5.3
2,-74.004951,40.748075,-74.013482,40.715892,1,9.5
3,-73.988091,40.733528,-73.939537,40.705488,3,17.5
4,-73.970687,40.764815,-73.984393,40.764038,5,5.3


In [3]:
FEATURE_COLS = np.arange(0,5)
TARGET_COL   = 'fare_amount'

In [4]:
def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual-predicted)**2))

def print_rmse(model):
  print "Train RMSE = {0}".format(compute_rmse(df_train[TARGET_COL], model.predict(df_train.iloc[:,FEATURE_COLS].values)))
  print "Valid RMSE = {0}".format(compute_rmse(df_valid[TARGET_COL], model.predict(df_valid.iloc[:,FEATURE_COLS].values)))

<h3> DNN using tf.layers </h3>

In [5]:
predictors = df_train.iloc[:,FEATURE_COLS].values
targets = df_train[TARGET_COL].values
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
modelprefix = 'taxi_model'
with tf.Session() as sess:
  npredictors = len(FEATURE_COLS)
  noutputs = 1
  feature_data = tf.placeholder("float", [None, npredictors])
  target_data = tf.placeholder("float", [None, noutputs])
  hidden = tf.contrib.layers.stack(feature_data, 
                                  tf.contrib.layers.fully_connected, 
                                  [64, 8],
                                  activation_fn=tf.nn.relu,
                                  biases_initializer=tf.ones,
                                  scope='fc')
  model = tf.contrib.layers.fully_connected(hidden, 1, activation_fn=None)
  cost = tf.nn.l2_loss(model - target_data)
  tf.scalar_summary("cost", cost)
  training_step = tf.contrib.layers.optimize_loss(cost, 
                                                  tf.contrib.framework.get_global_step(),
                                                  optimizer='Adam', learning_rate=0.1)

  summary_writer = tf.train.SummaryWriter('taxi_model',graph=sess.graph)

  tf.initialize_all_variables().run()
  merged_summary_op = tf.merge_all_summaries()
  for iter in xrange(0, 1000):
    _, trainerr, summary = sess.run([training_step, cost, merged_summary_op], feed_dict = {
        feature_data : predictors,
        target_data : targets.reshape(len(predictors), noutputs)
      })
    summary_writer.add_summary(summary)
    if (iter%100 == 1):
      trmse = np.sqrt(trainerr/len(predictors))     
      print 'iter={0} train_error={1}'.format(iter, trmse)

iter=1 train_error=10.3469288494
iter=101 train_error=7.3345376958
iter=201 train_error=6.60928236778
iter=301 train_error=6.57825321644
iter=401 train_error=6.57793345164
iter=501 train_error=6.57793131167
iter=601 train_error=6.57793498019
iter=701 train_error=6.57793498019
iter=801 train_error=6.57793498019
iter=901 train_error=6.57793498019


In [6]:
%tensorboard start --logdir=taxi_model

In [ ]:
%tensorboard stop --pid=4554